In [1]:
import sys, os #imp
sys.path.append('..')
import pandas as pd
import yaml
import backtesting as bt
import matplotlib.pyplot as plt

from tools.instruments import instruments

In [2]:
plt.ioff()

In [3]:
with open('systems/emacross.yaml', encoding='utf-8') as f:
    system1 = yaml.load(f, Loader=yaml.FullLoader)
    print(system1)

{'name': 'emacross', 'description': '시스템 설명', 'sectors': 'default', 'instruments': ['AD', 'CL', 'C', 'NQ', 'GC', 'BO'], 'from_date': '2002-01-01', 'to_date': '2004-01-01', 'principal': 100000, 'heat_system': 'DefaultHeat', 'max_system_heat': 0.4, 'max_sector_heat': 0.4, 'max_trade_heat': 0.1, 'max_lots': 2, 'commission': 2.99, 'skid': 0.25, 'metrics': [['ema200', 'EMA', 'window=200'], ['ma20', 'MA', 'window=20'], ['ma200', 'MA', 'window=200'], ['volatility', 'ATR', 'window=30'], ['min30', 'MIN', 'window=30'], ['max20', 'MAX', 'window=20']], 'entry_rule': {'long': 'ma20>ma200', 'short': None}, 'exit_rule': {'long': 'ma20<ma200', 'short': None}, 'stop_rule': {'long': 'min30', 'short': None}}


In [4]:
trader = bt.Trader(systems=[system1])
system = trader.systems[0]

In [5]:
trader.run()

매매시작
거래일: 2002-01-02 00:00:00, 시스템: emacross 2.5501925945281982sec
거래일: 2002-01-03 00:00:00, 시스템: emacross 0.00508570671081543sec
거래일: 2002-01-04 00:00:00, 시스템: emacross 0.003989219665527344sec
거래일: 2002-01-07 00:00:00, 시스템: emacross 0.003989219665527344sec
거래일: 2002-01-08 00:00:00, 시스템: emacross 0.005983829498291016sec
거래일: 2002-01-09 00:00:00, 시스템: emacross 0.005014657974243164sec
거래일: 2002-01-10 00:00:00, 시스템: emacross 0.00498652458190918sec
거래일: 2002-01-11 00:00:00, 시스템: emacross 0.004987239837646484sec
거래일: 2002-01-14 00:00:00, 시스템: emacross 0.0049860477447509766sec
거래일: 2002-01-15 00:00:00, 시스템: emacross 0.004984140396118164sec
거래일: 2002-01-16 00:00:00, 시스템: emacross 0.004991769790649414sec
거래일: 2002-01-17 00:00:00, 시스템: emacross 0.004979372024536133sec
거래일: 2002-01-18 00:00:00, 시스템: emacross 0.0049915313720703125sec
거래일: 2002-01-21 00:00:00, 시스템: emacross 0.004986286163330078sec
거래일: 2002-01-22 00:00:00, 시스템: emacross 0.000997781753540039sec
거래일: 2002-01-23 00:00:00, 시스템: emacro

거래일: 2002-08-07 00:00:00, 시스템: emacross 0.007950782775878906sec
거래일: 2002-08-08 00:00:00, 시스템: emacross 0.007978439331054688sec
거래일: 2002-08-09 00:00:00, 시스템: emacross 0.0079803466796875sec
거래일: 2002-08-12 00:00:00, 시스템: emacross 0.007978677749633789sec
거래일: 2002-08-13 00:00:00, 시스템: emacross 0.006979465484619141sec
거래일: 2002-08-14 00:00:00, 시스템: emacross 0.007978677749633789sec
거래일: 2002-08-15 00:00:00, 시스템: emacross 0.007978677749633789sec
거래일: 2002-08-16 00:00:00, 시스템: emacross 0.007978677749633789sec
거래일: 2002-08-19 00:00:00, 시스템: emacross 0.006981372833251953sec
거래일: 2002-08-20 00:00:00, 시스템: emacross 0.0079803466796875sec
거래일: 2002-08-21 00:00:00, 시스템: emacross 0.0069811344146728516sec
거래일: 2002-08-22 00:00:00, 시스템: emacross 0.00797724723815918sec
거래일: 2002-08-23 00:00:00, 시스템: emacross 0.0079803466796875sec
거래일: 2002-08-26 00:00:00, 시스템: emacross 0.007978439331054688sec
거래일: 2002-08-27 00:00:00, 시스템: emacross 0.006981372833251953sec
거래일: 2002-08-28 00:00:00, 시스템: emacross 0.0069

거래일: 2003-02-26 00:00:00, 시스템: emacross 0.005982398986816406sec
거래일: 2003-02-27 00:00:00, 시스템: emacross 0.0069963932037353516sec
거래일: 2003-02-28 00:00:00, 시스템: emacross 0.005969047546386719sec
거래일: 2003-03-03 00:00:00, 시스템: emacross 0.005984067916870117sec
거래일: 2003-03-04 00:00:00, 시스템: emacross 0.005983829498291016sec
거래일: 2003-03-05 00:00:00, 시스템: emacross 0.005983829498291016sec
거래일: 2003-03-06 00:00:00, 시스템: emacross 0.004987001419067383sec
거래일: 2003-03-07 00:00:00, 시스템: emacross 0.0059854984283447266sec
거래일: 2003-03-10 00:00:00, 시스템: emacross 0.005983829498291016sec
거래일: 2003-03-11 00:00:00, 시스템: emacross 0.004986763000488281sec
거래일: 2003-03-12 00:00:00, 시스템: emacross 0.005984067916870117sec
거래일: 2003-03-13 00:00:00, 시스템: emacross 0.00498652458190918sec
거래일: 2003-03-14 00:00:00, 시스템: emacross 0.005982875823974609sec
거래일: 2003-03-17 00:00:00, 시스템: emacross 0.008975744247436523sec
거래일: 2003-03-18 00:00:00, 시스템: emacross 0.008975744247436523sec
거래일: 2003-03-19 00:00:00, 시스템: emacross

거래일: 2003-09-03 00:00:00, 시스템: emacross 0.006981611251831055sec
거래일: 2003-09-04 00:00:00, 시스템: emacross 0.006981372833251953sec
거래일: 2003-09-05 00:00:00, 시스템: emacross 0.005983591079711914sec
거래일: 2003-09-08 00:00:00, 시스템: emacross 0.006981372833251953sec
거래일: 2003-09-09 00:00:00, 시스템: emacross 0.006981372833251953sec
거래일: 2003-09-10 00:00:00, 시스템: emacross 0.00797891616821289sec
거래일: 2003-09-11 00:00:00, 시스템: emacross 0.007978439331054688sec
거래일: 2003-09-12 00:00:00, 시스템: emacross 0.006981372833251953sec
거래일: 2003-09-15 00:00:00, 시스템: emacross 0.006982326507568359sec
거래일: 2003-09-16 00:00:00, 시스템: emacross 0.007978439331054688sec
거래일: 2003-09-17 00:00:00, 시스템: emacross 0.006981372833251953sec
거래일: 2003-09-18 00:00:00, 시스템: emacross 0.006981372833251953sec
거래일: 2003-09-19 00:00:00, 시스템: emacross 0.006981372833251953sec
거래일: 2003-09-22 00:00:00, 시스템: emacross 0.008975982666015625sec
거래일: 2003-09-23 00:00:00, 시스템: emacross 0.007977724075317383sec
거래일: 2003-09-24 00:00:00, 시스템: emacross 0

In [6]:
system.create_report()

In [11]:
system.equity.cash

-311640.7